In [0]:
%run ./0

In [0]:
##컬럼
all_col=['host_is_superhost','cancellation_policy','instant_bookable','host_total_listings_count','neighbourhood_cleansed','latitude','longitude','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','minimum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','target_price','index']

fs = FeatureStoreClient()

##feature store에 등록하려는 id 생성
table_name = f"airbnb_db.feature_store_" + str(uuid.uuid4())[:6]
feature_df=data_reading_table_select_col('airbnb_db.gold_final_mart_table_updated_airbnb',str(all_col[:])[1:-1]).drop('target_price')
schema = feature_df.schema
description = "feature_store_feature_all"

##feature store에 feature table 등록
fs.create_table(
    name=table_name,
    primary_keys = ["index"],
    df=feature_df,
    schema=feature_df.schema,
    description = description
)
##자동화하기 위해 feature store로 등록된 id들 airbnb_db.feature_store_tb이라는 테이블에서 관리 
##이런 방식으로 최신 feature store에 등록된 id 가지고 데이터 로딩 가능
insert=spark.sql(f'''insert into airbnb_db.feature_store_tb (feature_id,created_timestamp) values("{table_name}","{datetime.datetime.now()}")''')
df=spark.sql('''select * from airbnb_db.feature_store_tb order by created_timestamp desc limit 1''');
df.display()

2023/03/16 06:15:46 WARNING databricks.feature_store.utils.spark_listener: Exception while attempting to initialize JVM-side state for logging Spark datasource. Either use a cluster with ML Runtime or install the mlflow-spark JAR in your cluster as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental.
2023/03/16 06:15:51 INFO databricks.feature_store._compute_client._compute_client: Created feature table 'hive_metastore.airbnb_db.feature_store_e4fa7b'.


feature_id,created_timestamp
airbnb_db.feature_store_e4fa7b,2023-03-16T06:15:52.386+0000
